# Regression using the hetero graphs

- Predicting income, property values, and rents using the sociodemographics and mobility networks.
- Three regressions: only sociodemographics, only mobility networks, and mobility + sociodemographics. LASSO is used when the input variable size is large.
- IMPORTANT: adjusting the right ALPHA is the key! Use the alpha associated around 15 non-zero coefficients, otherwise the results are WRONG.

TBD
- Make the normalization method consistent.
- Automate the choice of lambda, now it is still a handcrafted process.
- Refine the lambda values to further improve the performance.


In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import networkx as nx
import matplotlib.pyplot as plt
import pickle
import copy
import scipy.sparse as sp
from scipy.sparse import csr_matrix
from sklearn.linear_model import LinearRegression

# regression tools
import statsmodels.api as sm
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler


Bad key text.latex.unicode in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key text.latex.preview in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 125 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 157 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso

In [3]:
# read files
with open("../../data/02_intermediate/socioecon_boston_miami_chicago_nyc_ct_shp_dic.pickle", 'rb') as f:
    socioecon_shp_dic = pickle.load(f)

# hetero activity graphs    
with open("../../data/03_processed/A_home_activity_three_cities_unweighted_dic.pickle", 'rb') as f:
    A_home_unweighted_dic = pickle.load(f)

with open("../../data/03_processed/A_home_activity_three_cities_weighted_dic.pickle", 'rb') as f:
    A_home_weighted_dic = pickle.load(f)


In [4]:
for col in socioecon_shp_dic['boston'].columns:
    print(col)

STATEFP
COUNTYFP
TRACTCE
NAME
NAMELSAD
MTFCC
FUNCSTAT
ALAND
AWATER
INTPTLAT
INTPTLON
geometry
pop_total_2018
sex_total_2018
sex_male_2018
sex_female_2018
age_median_2018
households_2018
race_total_2018
race_white_2018
race_black_2018
race_native_2018
race_asian_2018
inc_total_pop_2018
inc_no_pop_2018
inc_with_pop_2018
inc_pop_10k_2018
inc_pop_1k_15k_2018
inc_pop_15k_25k_2018
inc_pop_25k_35k_2018
inc_pop_35k_50k_2018
inc_pop_50k_65k_2018
inc_pop_65k_75k_2018
inc_pop_75k_2018
inc_median_ind_2018
travel_total_to_work_2018
travel_driving_to_work_2018
travel_pt_to_work_2018
travel_taxi_to_work_2018
travel_cycle_to_work_2018
travel_walk_to_work_2018
travel_work_from_home_2018
edu_total_pop_2018
bachelor_male_25_34_2018
master_phd_male_25_34_2018
bachelor_male_35_44_2018
master_phd_male_35_44_2018
bachelor_male_45_64_2018
master_phd_male_45_64_2018
bachelor_male_65_over_2018
master_phd_male_65_over_2018
bachelor_female_25_34_2018
master_phd_female_25_34_2018
bachelor_female_35_44_2018
master_

In [5]:
# initialize performance and coefficients dictionaries
performance_dic = {}
coeff_dic = {}

In [6]:
# parameters
threshold = 1.0 # 1.0, 10.0. - Using only the 1.0 threshold.
top_k = 15 # save the top 15 coefficients for the LASSO results.

# Three cities

- I ran the scripts nine times (3 * 3) by HANDS! This process needs to be automated!
- Three cities * Three output variables

In [7]:
# Set up parameters
city = 'boston'
# city = 'chicago'
# city = 'miami'

# 
performance_dic[city] = {}
coeff_dic[city] = {}

In [8]:
# working on individual cities
# augment the activity data to sociodemographics
df_socioecon_shp = socioecon_shp_dic[city]

for activity in A_home_unweighted_dic[city][threshold].keys():
    activity_vector = pd.DataFrame(A_home_unweighted_dic[city][threshold][activity].sum(axis = 1), columns = [activity])
    df_socioecon_shp = df_socioecon_shp.merge(activity_vector, 
                               left_index = True, 
                               right_index = True, 
                               how = 'left')

df_socioecon_shp.fillna(0.0, inplace=True)
df_socioecon_shp


,STATEFP,COUNTYFP,TRACTCE,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,...,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings,Women's Store,Yoga Studio,Yogurt,Zoo,Zoo Exhibit
25009201100,25,009,201100,2011,Census Tract 2011,G5020,S,2786306,3481257,+42.4282342,...,0.0,3.0,3.0,1.0,1.0,5.0,5.0,2.0,0.0,0.0
25009202101,25,009,202101,2021.01,Census Tract 2021.01,G5020,S,1454942,7226,+42.4758943,...,0.0,3.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
25009202102,25,009,202102,2021.02,Census Tract 2021.02,G5020,S,2919051,24048,+42.4829911,...,0.0,2.0,0.0,1.0,1.0,6.0,1.0,2.0,0.0,0.0
25009202200,25,009,202200,2022,Census Tract 2022,G5020,S,3452798,1441782,+42.4695768,...,1.0,3.0,0.0,1.0,2.0,7.0,3.0,5.0,0.0,0.0
25009203100,25,009,203100,2031,Census Tract 2031,G5020,S,3720285,1481336,+42.4871332,...,1.0,6.0,4.0,0.0,1.0,4.0,5.0,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33017085000,33,017,085000,850,Census Tract 850,G5020,S,120890933,5035519,+43.2155657,...,0.0,0.0,0.0,1.0,3.0,5.0,4.0,2.0,0.0,0.0
33017086000,33,017,086000,860,Census Tract 860,G5020,S,126932596,5722104,+43.2753082,...,1.0,0.0,3.0,1.0,2.0,3.0,4.0,3.0,0.0,0.0
33017087000,33,017,087000,870,Census Tract 870,G5020,S,94758837,909784,+43.3669235,...,0.0,1.0,1.0,1.0,2.0,3.0,0.0,3.0,0.0,0.0
33017088000,33,017,088000,880,Census Tract 880,G5020,S,85593614,3177277,+43.4522291,...,0.0,2.0,1.0,1.0,3.0,5.0,2.0,3.0,0.0,0.0


## Three outputs

In [36]:
# output variables
# output_var = 'inc_median_household_2018'
output_var = 'property_value_median_2018' 
# output_var = 'rent_median_2018' 

# Create the dictionaries to save the results.
performance_dic[city][output_var] = {}
coeff_dic[city][output_var] = {}


#### 1. only mobility networks

In [37]:
# assigne inputs and outputs
var_list_1 = list(A_home_unweighted_dic[city][threshold].keys())

var_2 = output_var

X = df_socioecon_shp[var_list_1]
X = sm.add_constant(X)
X = pd.DataFrame(normalize(X, axis = 0), columns = X.columns) # normalize 
y = normalize(df_socioecon_shp[var_2].values.reshape(-1, 1), axis = 0) # normalize 

# y = df_socioecon_shp[var_2].values.reshape(-1, 1)


# split training and testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

# searching in L1 regularized regressions
alpha_list = [0.00, 0.000001, 0.000002, 0.000005, 0.00001, 0.00002, 0.00003, 0.00004, 0.00005, 0.0001, 0.001, 0.01, 0.1, 1.0]
# alpha_list = [0.00, 0.01, 0.1, 1.0, 5.0, 10.0, 20.0, 50.0, 100, 200]

performance_table = pd.DataFrame(np.zeros((3, len(alpha_list))),
                                 index = ['training', 'testing', 'coef counts'],
                                 columns = alpha_list)

for alpha in alpha_list:
    model = Lasso(alpha = alpha)
    model.fit(X_train, y_train)    
    train_r2 = model.score(X_train, y_train)
    test_r2 = model.score(X_test, y_test)
    # count the non-zero coef
    pos_counts = np.sum(model.coef_ > 0.01)
    neg_counts = np.sum(model.coef_ < -0.01)
    total_counts = pos_counts + neg_counts
    
    performance_table.loc['training', alpha] = train_r2
    performance_table.loc['testing', alpha] = test_r2
    performance_table.loc['coef counts', alpha] = int(total_counts)

/home/jtl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
/home/jtl/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/home/jtl/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.098e-03, tolerance: 1.229e-05 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  coef_, l1_reg, l2_reg, X, y, max_i

In [38]:
performance_table

,0.000000,0.000001,0.000002,0.000005,0.000010,0.000020,0.000030,0.000040,0.000050,0.000100,0.001000,0.010000,0.100000,1.000000
training,0.982130,0.923540,0.886652,0.812420,0.740526,0.637455,0.564056,0.491934,0.433001,0.203217,0.00000,0.00000,0.00000,0.00000
testing,-4.958179,0.254311,0.434445,0.531198,0.545673,0.519486,0.479393,0.426121,0.373262,0.146165,-0.00022,-0.00022,-0.00022,-0.00022
coef counts,554.000000,249.000000,176.000000,108.000000,67.000000,31.000000,22.000000,15.000000,8.000000,1.000000,0.00000,0.00000,0.00000,0.00000


In [39]:
# Rerun the model to find the important coefficients
alpha = 0.00004
# alpha = 50.0
model = Lasso(alpha = alpha)
model.fit(X_train, y_train)   

# col names 
full_col_names = ['const'] + var_list_1

# combine coefficients and names
coef = pd.DataFrame(model.coef_, index = full_col_names, columns = ['value'])
pd.set_option('display.max_rows', 500)
coef.sort_values('value')

,value
Discount Store,-0.071985
Tattoo,-0.048903
Caribbean,-0.023017
Auto Garage,-0.016270
Hot Dogs,-0.012075
...,...
Art Museum,0.024919
Juice Bar,0.029682
Cycle Studio,0.038201
Salad,0.053561


In [40]:
# save performance and coefficients
performance_dic[city][output_var]['lasso (no socio-demographics)'] = performance_table

# save coefficients
top_k_index = np.abs(coef).sort_values('value').iloc[-top_k:,:].index
top_k_coefficients = coef.loc[top_k_index, :]
top_k_coefficients_sorted = top_k_coefficients.sort_values('value')
coeff_dic[city][output_var]['lasso (no socio-demographics)'] = top_k_coefficients_sorted


In [41]:
top_k_coefficients_sorted

,value
Discount Store,-0.071985
Tattoo,-0.048903
Caribbean,-0.023017
Auto Garage,-0.016270
Hot Dogs,-0.012075
Cantonese,-0.011509
Storage,-0.010528
Climbing Gym,0.011105
Synagogue,0.011850
Watch Shops,0.013405


#### 2. only sociodemographics

In [42]:
# create the baseline regressions
var_list_1 = ['pop_total_2016',
              'pop_density_2016',
              'age_median_2016', 
              'household_size_avg_2016',
              'sex_male_ratio_2016', 
              'race_white_ratio_2016',
              'edu_higher_edu_ratio_2016',            
              'vehicle_per_capita_2016']

# predict inc_per_capita_2018
var_2 = output_var

X = df_socioecon_shp[var_list_1]
X = pd.DataFrame(normalize(X, axis = 0), columns = X.columns)
X = sm.add_constant(X)
y = normalize(df_socioecon_shp[var_2].values.reshape(-1, 1), axis = 0)

# train test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

reg = LinearRegression().fit(X_train, y_train)
r2_train = np.round(reg.score(X_train, y_train), decimals = 3)
r2_test = np.round(reg.score(X_test, y_test), decimals = 3)
print("Training R2 is: ", r2_train)
print("Testing R2 is: ", r2_test)

mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

Training R2 is:  0.614
Testing R2 is:  0.522
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.591
Model:                            OLS   Adj. R-squared:                  0.588
Method:                 Least Squares   F-statistic:                     177.9
Date:                Sun, 10 Jul 2022   Prob (F-statistic):          3.53e-185
Time:                        10:51:51   Log-Likelihood:                 3329.5
No. Observations:                 992   AIC:                            -6641.
Df Residuals:                     983   BIC:                            -6597.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------

In [43]:
# save results
performance_dic[city][output_var]['baseline'] = pd.DataFrame([r2_train, r2_test, len(var_list_1)], 
                                                              index = ['training', 'testing', 'coef counts'],
                                                              columns = ['baseline'])


#### 3. combining activities and sociodemographics

In [44]:
var_list_1 = ['pop_total_2016',
              'pop_density_2016',
              'age_median_2016', 
              'household_size_avg_2016',
              'sex_male_ratio_2016', 
              'race_white_ratio_2016',
              'edu_higher_edu_ratio_2016',            
               'vehicle_per_capita_2016']

var_list_1 += list(A_home_unweighted_dic[city][threshold].keys())
# var_list_1 += joint_activity_list

# print(len(var_list_1))

# var_2_list = ['inc_per_capita_2018', 'property_value_median_2018']
var_2 = output_var

# scaler = StandardScaler()
# X = df_socioecon_shp[var_list_1]
# # X = sm.add_constant(X)
# X = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)
# y = scaler.fit_transform(df_socioecon_shp[var_2].values.reshape(-1, 1))

X = df_socioecon_shp[var_list_1]
X = sm.add_constant(X)
X = pd.DataFrame(normalize(X, axis = 0), columns = X.columns)
y = normalize(df_socioecon_shp[var_2].values.reshape(-1, 1), axis = 0)

# split training and testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

# searching in L1 regularized regressions
alpha_list = [0.00, 0.0000001, 0.0000005, 0.000001, 0.000002, 0.000005, 0.00001, 0.00002, 0.00005, 0.01, 0.1, 1.0]
# alpha_list = [0.00, 0.1, 1.0, 5.0, 10.0, 20.0, 50.0, 100, 200]

performance_table = pd.DataFrame(np.zeros((3, len(alpha_list))),
                                 index = ['training', 'testing', 'coef counts'],
                                 columns = alpha_list)

for alpha in alpha_list:
    model = Lasso(alpha = alpha)
    model.fit(X_train, y_train)
    train_r2 = model.score(X_train, y_train)
    test_r2 = model.score(X_test, y_test)
    # count the non-zero coef
    pos_counts = np.sum(model.coef_ > 0.01)
    neg_counts = np.sum(model.coef_ < -0.01)
    total_counts = pos_counts + neg_counts
    
    performance_table.loc['training', alpha] = train_r2
    performance_table.loc['testing', alpha] = test_r2
    performance_table.loc['coef counts', alpha] = int(total_counts)

/home/jtl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
/home/jtl/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/home/jtl/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.045e-04, tolerance: 1.229e-05 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  coef_, l1_reg, l2_reg, X, y, max_i

In [45]:
performance_table

,0.000000e+00,1.000000e-07,5.000000e-07,1.000000e-06,2.000000e-06,5.000000e-06,1.000000e-05,2.000000e-05,5.000000e-05,1.000000e-02,1.000000e-01,1.000000e+00
training,0.988538,0.983802,0.960039,0.934969,0.900042,0.834789,0.774699,0.688505,0.531279,0.00000,0.00000,0.00000
testing,-4.032309,-1.013365,0.099530,0.405626,0.543646,0.612925,0.626647,0.613522,0.493311,-0.00022,-0.00022,-0.00022
coef counts,542.000000,442.000000,311.000000,228.000000,151.000000,77.000000,45.000000,19.000000,5.000000,0.00000,0.00000,0.00000


In [48]:
# Rerun the model to find the important coefficients
# alpha = 50.0
alpha = 4.000000e-05
model = Lasso(alpha = alpha)
model.fit(X_train, y_train)   

# col names 
full_col_names = ['const'] + var_list_1
# full_col_names = var_list_1

# combine coefficients and names
coef = pd.DataFrame(model.coef_, index = full_col_names, columns = ['value'])
pd.set_option('display.max_rows', 700)
coef.sort_values('value')


,value
Tattoo,-0.019044
Stationery Store,-0.008400
Supplement Shop,-0.005814
Credit Union,-0.004488
Video Games,-0.003427
Storage,-0.002368
Cantonese,-0.002143
Auto Garage,-0.001974
Hot Dogs,-0.000647
Outdoor Supplies,-0.000000


In [49]:
# save performance and coefficients
performance_dic[city][output_var]['lasso (with socio-demographics)'] = performance_table

# save coefficients
top_k_index = np.abs(coef).sort_values('value').iloc[-top_k:,:].index
top_k_coefficients = coef.loc[top_k_index, :]
top_k_coefficients_sorted = top_k_coefficients.sort_values('value')
coeff_dic[city][output_var]['lasso (with socio-demographics)'] = top_k_coefficients_sorted


In [50]:
top_k_coefficients_sorted

,value
Tattoo,-0.019044
Stationery Store,-0.008400
Supplement Shop,-0.005814
Credit Union,-0.004488
Video Games,-0.003427
Storage,-0.002368
Cantonese,-0.002143
Juice Bar,0.002810
pop_density_2016,0.002978
Line / Queue,0.005501


In [51]:
top_k_coefficients_sorted

,value
Tattoo,-0.019044
Stationery Store,-0.008400
Supplement Shop,-0.005814
Credit Union,-0.004488
Video Games,-0.003427
Storage,-0.002368
Cantonese,-0.002143
Juice Bar,0.002810
pop_density_2016,0.002978
Line / Queue,0.005501


# Save

- Don't save until the 3 * 3 = 9 cases are all completed!!

In [101]:
###
with open('../../data/05_model_outputs/lasso_performance.pickle', 'wb') as f:
    pickle.dump(performance_dic, f)
    
with open('../../data/05_model_outputs/lasso_coefficients.pickle', 'wb') as f:
    pickle.dump(coeff_dic, f)
